In [ ]:
#Codigo de importaciones
#Importaciones necesarias para que funcione el codigo
import pandas as pd
import pyreadr
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast  # Para convertir strings seguros a listas
import re


In [ ]:
#Codigo de creación de carpetas
# Nombre de la carpeta donde antes de ejecutar el script se guardan los datos CaliBRANDO2024.dta
carpeta = "insumos"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

carpeta = "insumos/data"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

carpeta = "insumos/marco"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

carpeta = "insumos/tablas_cruzadas"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

# Nombre de la carpeta donde se guardaran los datos
carpeta = "outputs"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

carpeta = "outputs/tablas_latex"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

In [ ]:
#Se lee la base de datos
#df = pd.read_stata("insumos/Panel 2023_ 1.dta")  # Leer el archivo .dta
df = pd.read_csv("insumos/data/data_&_variables_nuevas_Calipedia.csv", sep=";", low_memory=False)


In [ ]:
#Se lee la base de datos de los insumos para las tablas
insumoTablas = pd.read_excel("insumos/tablas_cruzadas/TablasCalipedia.xlsx")

In [ ]:
#Se muestran los labels de las variables
# Cargar el archivo con un iterador
data_reader = pd.read_stata("insumos/data/Panel 2023_ 1.dta", iterator=True)

# Extraer las etiquetas de las variables
variable_labels = data_reader.variable_labels()

# Agregar las etiquetas manualmente como un atributo del DataFrame
df.attrs["variable_labels"] = variable_labels

# Ver las etiquetas
print(df.attrs["variable_labels"])

In [ ]:
df

In [ ]:
#Se ve información por año
# Contar valores y convertir a DataFrame
df_counts = df["P01000801"].value_counts().reset_index()

# Renombrar columnas
df_counts.columns = ["P01000801", "Numero de encuestas"]

# Convertir los años a enteros
df_counts["P01000801"] = df_counts["P01000801"].astype(int)

# Ordenar por P01000801 en orden descendente
df_counts = df_counts.sort_values(by="P01000801", ascending=False).reset_index(drop=True)

# Filtrar para mostrar solo de 2023 a 2015
df_counts = df_counts[df_counts["P01000801"].between(2015, 2023)]

# Mostrar como tabla
df_counts


In [ ]:
# Agrupar por Masterclass y luego por Clase
grupo = insumoTablas.groupby(["Masterclas", "Clase"]).size()

# Imprimir resultado
print(grupo)


# Posibles procesos necesarios?

In [ ]:
df["P02001401"].unique()

In [ ]:
# Mapeo de valores
mapeo = {
    'primaria completa': 'Primaria',
    'secundaria completa': 'Secundaria',
    'Técnica o Técnológica': 'Técnico',
    'Profesional': 'Profesional',
    'Especialización': 'Posgrado',
    'Maestría o Doctorado': 'Posgrado',
    'Ninguno': 'Ninguno'
}

# Reemplazar valores según el mapeo y convertir incompletos en NaN
df["P02001401"] = df["P02001401"].replace(mapeo)
df["P02001401"] = df["P02001401"].where(df["P02001401"].isin(mapeo.values()), np.nan)


In [ ]:
df["P03000101"] = df["P03000101"].map(lambda x: 10.0 if x == "Totalmente Satisfecho" 
                                      else 0.0 if x == "Nada Satisfecho" 
                                      else x)

# Convertimos todo a float (para que los valores '1.0', '2.0', etc., no sean strings)
df["P03000101"] = pd.to_numeric(df["P03000101"], errors='coerce')


In [ ]:
df["P03001601"] = df["P03001601"].map(lambda x: 10.0 if x == "Mejor vida posible" 
                                      else 0.0 if x == "Peor vida posible" 
                                      else x)

# Convertimos todo a float
df["P03001601"] = pd.to_numeric(df["P03001601"], errors='coerce')


In [ ]:
#Mapeos necesarios para que las tablas se vean bien
# Diccionario de mapeo
nse_labels = {1: "Bajo", 2: "Medio", 3: "Alto"}
formalidad_labels = {0: "Informal", 1: "Formal"}
minoriaetnica_labels = {0: "No", 1: "Si"}
                                                               

# Aplicar mapeo a las columnas
df["NSE"] = df["NSE"].map(nse_labels)
df["formalidad"] = df["formalidad"].map(formalidad_labels)
df["minoriaetnica"] = df["minoriaetnica"].map(minoriaetnica_labels)



In [ ]:
df["P01000801"] = df["P01000801"].astype(int)


In [ ]:
df = df.rename(columns={"balance_afectivo": "balanceAfectivo"})

# Codigo de tablas cruzadas

In [ ]:
insumoTablas

In [ ]:
def generate_table_porcentaje_lista_porcentaje_lista_con_formula(df, fila, columnas, total_columna):
    resultado = {}
    for valor in df[fila].unique():
        df_filtrado = df[df[fila] == valor] 
        if df_filtrado[total_columna].sum() == 0:
            continue 
        porcentajes = {col: round((df_filtrado[col] / df_filtrado[total_columna]).mean() * 100, 1) for col in columnas}
        resultado[valor] = porcentajes
    resultado_df = pd.DataFrame.from_dict(resultado, orient='index')
    resultado_df.index.name = fila
    return resultado_df

def generate_table_doble_entrada_lista_con_formula(df, fila, columnas, total_columna):
    cat_col = columnas[0]  # columna categórica secundaria
    num_cols = columnas[1:]  # columnas numéricas a dividir

    df_copy = df.copy()
    for col in num_cols:
        df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
    df_copy[total_columna] = pd.to_numeric(df_copy[total_columna], errors='coerce')

    # Calcular proporciones
    for col in num_cols:
        df_copy[f"{col}"] = df_copy[col] / df_copy[total_columna] * 100

    # Agrupar y promediar proporciones
    prop_cols = [f"{col}" for col in num_cols]
    resultado = df_copy.groupby([fila, cat_col])[prop_cols].mean().round(1).reset_index()

    return resultado

    

def generate_table_porcentaje(df, fila, columna=None):
    if columna is None:
        tabla_frecuencia = df[fila].value_counts().to_frame()
        tabla_frecuencia.columns = ['Frecuencia']
        tabla_frecuencia["Porcentaje"] = round((tabla_frecuencia["Frecuencia"] / tabla_frecuencia["Frecuencia"].sum()) * 100, 1)
        return tabla_frecuencia
    else:
        tabla_frecuencia = pd.crosstab(df[fila], df[columna])
        tabla_porcentaje = round(tabla_frecuencia.div(tabla_frecuencia.sum(axis=1), axis=0) * 100, 1)
        return tabla_porcentaje

def generate_table_porcentaje_lista(df, fila, columnas):
    resultado = {}
    df_copy = df.copy()
    for col in columnas:
        if df_copy[col].dtype == 'object':
            valores_unicos = df_copy[col].dropna().unique()
            if set(valores_unicos).issubset({"Sí", "Si", "No"}):
                df_copy[col] = df_copy[col].replace({"Sí": 1, "Si": 1, "No": 0})
    df_copy[columnas] = df_copy[columnas].apply(pd.to_numeric, errors='coerce')
    for valor in df_copy[fila].dropna().unique():
        df_filtrado = df_copy[df_copy[fila] == valor]
        suma_si = df_filtrado[columnas].sum()
        total_respuestas = df_filtrado[columnas].count()
        porcentajes = round((suma_si / total_respuestas) * 100, 1)
        resultado[valor] = porcentajes.to_dict()
    resultado_df = pd.DataFrame.from_dict(resultado, orient='index')
    resultado_df.index.name = fila
    return resultado_df

def generate_table_promedio_columna(df, fila, columna):
    if not pd.api.types.is_numeric_dtype(df[columna]):
        raise ValueError(f"La columna '{columna}' debe ser numérica para calcular el promedio.")
    resultado = df.groupby(fila)[columna].mean().to_frame()
    resultado.columns = [f"Promedio de {columna}"]
    return round(resultado, 1)

def generate_table_promedio_columna_lista(df, fila, columnas):
    columnas_validas = []
    df_copy = df.copy()
    for col in columnas:
        try:
            df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
            if df_copy[col].notna().sum() > 0:
                columnas_validas.append(col)
        except Exception:
            pass
    if not columnas_validas:
        raise ValueError("Ninguna de las columnas seleccionadas se pudo convertir a numérica.")
    resultado = df_copy.groupby(fila)[columnas_validas].mean()
    resultado.columns = [f"Promedio de {col}" for col in resultado.columns]
    return round(resultado, 1)

def generate_table_doble_entrada(df, fila, columnas):
    if len(columnas) != 2:
        raise ValueError("Debes proporcionar exactamente dos columnas: una categórica y una numérica.")
    categoria = columnas[1]
    variable_numerica = columnas[0]
    df_copy = df.copy()
    df_copy[variable_numerica] = pd.to_numeric(df_copy[variable_numerica], errors='coerce')
    resultado = df_copy.groupby([fila, categoria])[variable_numerica].mean().reset_index()
    resultado.columns = [fila, categoria, f"Promedio de {variable_numerica}"]
    return round(resultado, 1)



#----------------------------------------------------------------------------------------------------------------
#Funciones para asegurar que los valores sean listas
def asegurar_lista(valor):
    if isinstance(valor, list):
        return valor  # Ya es una lista, la dejamos igual
    elif isinstance(valor, str):
        try:
            # Intenta evaluar el string como una lista real
            valor_evaluado = ast.literal_eval(valor)
            if isinstance(valor_evaluado, list):
                return valor_evaluado
            else:
                return [valor]  # Si no era una lista real, lo tratamos como string normal
        except (SyntaxError, ValueError):
            return [valor]  # Si no se puede evaluar, tratamos el string como un solo elemento
    else:
        return [valor]  # Para otros casos, encapsulamos en una lista
    


#----------------------------------------------------------------------------------------------------------------
def analisis_tablas(df, insumoTablas):
    tablas = []
    num_filas = insumoTablas.shape[0]
    
    ultima_masterclass = None
    contador_masterclass = 0  # Contador para Masterclass
    contador_clase = 0  # Contador para Clase

    clases_vistas = set()  # Conjunto para registrar combinaciones únicas de (Clase, Tipo)

    for i in range(num_filas):
        masterclass = insumoTablas.iloc[i]["Masterclas"]
        clase = insumoTablas.iloc[i]["Clase"]
        tipo = insumoTablas.iloc[i]["Tipo"]
        fila = insumoTablas.iloc[i]["Filas"]
        columnas = insumoTablas.iloc[i]["Columnas"]
        total_columna = insumoTablas.iloc[i]["total_columna"]

        if masterclass != ultima_masterclass:
            contador_masterclass += 1
            contador_clase = 0  # Reiniciar el contador de Clase para la nueva Masterclass
            clases_vistas.clear()  # Reiniciar el registro de clases vistas
            
            print("-" * 100)
            print(f"{contador_masterclass}. {masterclass.upper()}")
            print("-" * 100)

        # Si esta combinación ya se imprimió, la omitimos
        if (clase, tipo) not in clases_vistas:
            contador_clase += 1
            print(f"{contador_masterclass}.{contador_clase} - {clase} de tipo: {tipo}")
            clases_vistas.add((clase, tipo))  # Registrar la combinación como vista

        ultima_masterclass = masterclass  # Actualizar la última Masterclass procesada

        print("- Tabla numero:", i+1)
    return tablas


#----------------------------------------------------------------------------------------------------------------
#Funciones para generar las tablas
def generar_tablas(df, insumoTablas):
    tablas = []
    num_filas = insumoTablas.shape[0]
    
    ultima_masterclass = None
    contador_masterclass = 0  # Contador para Masterclass
    contador_clase = 0  # Contador para Clase

    clases_vistas = set()  # Conjunto para registrar combinaciones únicas de (Clase, Tipo)

    for i in range(num_filas):
        masterclass = insumoTablas.iloc[i]["Masterclas"]
        clase = insumoTablas.iloc[i]["Clase"]
        tipo = insumoTablas.iloc[i]["Tipo"]
        fila = insumoTablas.iloc[i]["Filas"]
        columnas = insumoTablas.iloc[i]["Columnas"]
        total_columna = insumoTablas.iloc[i]["total_columna"]

        if masterclass != ultima_masterclass:
            contador_masterclass += 1
            contador_clase = 0  # Reiniciar el contador de Clase para la nueva Masterclass
            clases_vistas.clear()  # Reiniciar el registro de clases vistas
            
            print("-" * 100)
            print(f"{contador_masterclass}. {masterclass.upper()}")
            print("-" * 100)

        # Si esta combinación ya se imprimió, la omitimos
        if (clase, tipo) not in clases_vistas:
            contador_clase += 1
            print(f"{contador_masterclass}.{contador_clase} - {clase} de tipo: {tipo}")
            clases_vistas.add((clase, tipo))  # Registrar la combinación como vista

        ultima_masterclass = masterclass  # Actualizar la última Masterclass procesada

        if pd.isna(columnas):  
            columnas = None

        tabla=None

        print("- Tabla numero:", i+1)
        if tipo == "porcentaje":
            tabla=generate_table_porcentaje(df, fila, columnas)
        if tipo == "promedio_columna":
            tabla=generate_table_promedio_columna(df, fila, columnas)

        columnas=asegurar_lista(columnas)

        if tipo == "promedio_columna_lista":
            tabla=generate_table_promedio_columna_lista(df, fila, columnas)
        if tipo == "porcentaje_lista":
            tabla=generate_table_porcentaje_lista(df, fila, columnas)
        if tipo == "porcentaje_lista_con_formula":
            tabla=generate_table_porcentaje_lista_porcentaje_lista_con_formula(df, fila, columnas, total_columna)
        if tipo == "doble_entrada_lista_con_formula":
            tabla=generate_table_doble_entrada_lista_con_formula(df, fila, columnas, total_columna)
        if tipo == "doble_entrada":
            tabla=generate_table_doble_entrada(df, fila, columnas)

        if tabla is not None:
            tablas.append(tabla)
            print(tabla)
        else:
            print("No se encontró el tipo de tabla:", tipo)
    return tablas


In [ ]:
analisis_tablas(df, insumoTablas)

In [ ]:
tablas=generar_tablas(df, insumoTablas)

In [ ]:
df['P02000701'].value_counts()

In [ ]:
numero_tabla=80
tabla=tablas[numero_tabla-1]
tabla


In [ ]:
def generar_tabla_latex(tabla, variable_labels):
    # Detectar códigos
    codigo_filas = tabla.index.name or "NO DETECTADO"
    codigo_columnas = tabla.columns.name or "NO DETECTADO"

    # Obtener etiquetas desde el diccionario
    etiqueta_filas = variable_labels.get(codigo_filas, "NO DETECTADO")
    etiqueta_columnas = variable_labels.get(codigo_columnas, "NO DETECTADO")

    num_columnas = tabla.shape[1]

    # Encabezado dinámico
    encabezado = r"""
\begin{table}[H]
\centering
\caption{"""+ f"{etiqueta_columnas} $\\sim$ {etiqueta_filas}" + r"""}
\begin{tabular}{l""" + "c" * num_columnas + r"""}
\toprule
& \multicolumn{""" + f"{num_columnas}" + r"""}{c}{\textbf{""" + codigo_columnas + r"""}} \\
\cmidrule(lr){2-""" + f"{num_columnas + 1}" + r"""}
\textbf{""" + codigo_filas + r"""} & """ + " & ".join(f"\\textbf{{{col}}}" for col in tabla.columns) + r""" \\
\midrule
"""

    cuerpo = ""
    for idx, fila in tabla.iterrows():
        fila_str = f"{idx} & " + " & ".join(f"{v:.1f}" for v in fila.values) + r" \\"
        cuerpo += fila_str + "\n"

    pie = r"""\bottomrule
\end{tabular}

\vspace{0.3cm}
\raggedright
\footnotesize
Elaboración propia con datos de CaliBRANDO 2014--2022. Resultados en porcentaje (\%). \\
\textsuperscript{""" + f"{codigo_columnas}" + r"""} """ + f"{etiqueta_columnas}" + r"""; \textsuperscript{""" + f"{codigo_filas}" + r"""} """ + f"{etiqueta_filas}" + r""".
\end{table}
"""

    return encabezado + cuerpo + pie


In [ ]:
def generar_tabla_latex2(tabla, variable_labels):
    # Detectar códigos
    codigo_columnas = tabla.columns.name or "NO DETECTADO"
    etiqueta_columnas = variable_labels.get(codigo_columnas, "NO DETECTADO")

    num_columnas = tabla.shape[1]

    # Encabezado sin columna para índice
    encabezado = r"""
\begin{table}[H]
\centering
\caption{""" + f"{etiqueta_columnas}" + r"""}
\begin{tabular}{""" + "c" * num_columnas + r"""}
\toprule
\multicolumn{""" + f"{num_columnas}" + r"""}{c}{\textbf{""" + codigo_columnas + r"""}} \\
\cmidrule(lr){1-""" + f"{num_columnas}" + r"""}
""" + " & ".join(f"\\textbf{{{col}}}" for col in tabla.columns) + r""" \\
\midrule
"""

    cuerpo = ""
    for _, fila in tabla.iterrows():
        valores_formateados = []
        for v in fila.values:
            try:
                num = float(v)
                if num.is_integer():
                    valores_formateados.append(f"{int(num)}")
                else:
                    valores_formateados.append(f"{num:.1f}")
            except (ValueError, TypeError):
                valores_formateados.append(str(v))
        fila_str = " & ".join(valores_formateados) + r" \\"
        cuerpo += fila_str + "\n"

    pie = r"""\bottomrule
\end{tabular}

\vspace{0.3cm}
\raggedright
\footnotesize
Elaboración propia con datos de CaliBRANDO 2014--2022. Resultados en porcentaje (\%). \\
\textsuperscript{""" + f"{codigo_columnas}" + r"""} """ + f"{etiqueta_columnas}" + r""".
\end{table}
"""

    return encabezado + cuerpo + pie


In [ ]:
try:
    latex_output = generar_tabla_latex(tabla, variable_labels)
    print(latex_output)
except Exception as e:
    #print("Error al generar la tabla en LaTeX:")
    #print(f"Detalles: {e}")
    latex_output = generar_tabla_latex2(tabla, variable_labels)
    print(latex_output)

In [ ]:
latex_total = ""  # Aquí acumularemos todo el contenido en LaTeX

for i, tabla in enumerate(tablas):
    try:
        latex_output = generar_tabla_latex(tabla, variable_labels)
    except Exception as e:
        # print(f"Error al generar la tabla {i} con la función principal. Usando función alternativa.")
        latex_output = generar_tabla_latex2(tabla, variable_labels)
    
    latex_total += f"% Tabla {i + 1}\n{latex_output}\n\n"

# Guardar en un archivo
with open("outputs/tablas_latex/tablas_latex.txt", "w", encoding="utf-8") as f:
    f.write(latex_total)
    print("Archivo 'tablas_latex.txt' guardado con éxito.")